In [1]:
from ease4lmp import BondedAtoms

## Create diamond crystal

In [2]:
from ase.build import bulk

We can use ASE's functionality to build crystal structure.
Here, we create an `ase.Atoms` instance for a diamond cube.

In [3]:
diamond_unit = bulk("C", "diamond", cubic=True)

In [4]:
diamond_unit

Atoms(symbols='C8', pbc=True, cell=[3.57, 3.57, 3.57])

In [5]:
diamond_unit.get_positions()

array([[0.    , 0.    , 0.    ],
       [0.8925, 0.8925, 0.8925],
       [0.    , 1.785 , 1.785 ],
       [0.8925, 2.6775, 2.6775],
       [1.785 , 0.    , 1.785 ],
       [2.6775, 0.8925, 2.6775],
       [1.785 , 1.785 , 0.    ],
       [2.6775, 2.6775, 0.8925]])

Next, we create a `ease4lmp.BondedAtoms` instace by inheriting from the created `ase.Atoms` instance.
For this purpose, we can use a static method `ease4lmp.BondedAtoms.inherit`.

In [6]:
diamond_unit_with_bonds = BondedAtoms.inherit(diamond_unit)

We calculate length of a bond between the nearest two carbon atoms. 

In [7]:
positions = diamond_unit_with_bonds.get_positions()
bond_vector = positions[1] - positions[0]
squared_bond_length = (bond_vector*bond_vector).sum()
squared_bond_length

2.38966875

Now, we start to compute and set bonds connecting carbon atoms.

To resolve a periodic boundary function, we create a $2 \times 2 \times 2$ supercell of `diamond_unit_with_bonds`.
Atoms in the supercell are tagged with original indices (indices in the original unit cell).

In [8]:
diamond_2x2x2 = diamond_unit_with_bonds.copy()
diamond_2x2x2.set_tags(list(range(len(diamond_2x2x2))))
diamond_2x2x2 *= 2

In [9]:
diamond_2x2x2.get_tags()

array([0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5,
       6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3,
       4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7])

We investigate vectors from each atom in `diamond_unit_with_bonds` to each atom in `diamond_2x2x2`.
If squred length of a vector is (nearly) equal to `squared_bond_length`, we store the following data:

* A two-element set consisting of index for the first and second atoms.
* An image flag of the unit cell where the second atom is in.

In [10]:
from math import isclose

cell_length = diamond_unit_with_bonds.get_cell()[1,1]

first_positions = diamond_unit_with_bonds.get_positions()

second_positions = diamond_2x2x2.get_positions()
original_index = diamond_2x2x2.get_tags()

stored_data = {}

for i, ri in enumerate(first_positions):
  for j, rj in zip(original_index, second_positions):
    
    vector = rj - ri
    
    if isclose((vector*vector).sum(), squared_bond_length, abs_tol=0.1):
      
      image_flag = tuple(map(int, rj/cell_length))
      
      stored_data[
        tuple(sorted([i, j])) # avoid duplication if both atoms are in the original unit cell.
        if image_flag == (0, 0, 0) else tuple([i, j])] = image_flag

In [11]:
stored_data

{(0, 1): (0, 0, 0),
 (1, 2): (0, 0, 0),
 (1, 4): (0, 0, 0),
 (1, 6): (0, 0, 0),
 (2, 3): (0, 0, 0),
 (3, 6): (0, 0, 1),
 (3, 4): (0, 1, 0),
 (3, 0): (0, 1, 1),
 (4, 5): (0, 0, 0),
 (5, 6): (0, 0, 1),
 (5, 2): (1, 0, 0),
 (5, 0): (1, 0, 1),
 (6, 7): (0, 0, 0),
 (7, 4): (0, 1, 0),
 (7, 2): (1, 0, 0),
 (7, 0): (1, 1, 0)}

Now, we set bonds to `diamond_unit_with_bonds`.

In [12]:
for k, v in stored_data.items():
  diamond_unit_with_bonds.add_bond(*k, img2=v)

Bond data is stored as a `numpy.ndarray` object.

In [13]:
diamond_unit_with_bonds.get_bonds()

array([[[ 1,  0,  0,  0],
        [ 3,  0, -1, -1],
        [ 5, -1,  0, -1],
        [ 7, -1, -1,  0]],

       [[-1,  0,  0,  0],
        [ 1,  0,  0,  0],
        [ 3,  0,  0,  0],
        [ 5,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [ 1,  0,  0,  0],
        [ 3, -1,  0,  0],
        [ 5, -1,  0,  0]],

       [[-1,  0,  0,  0],
        [ 3,  0,  0,  1],
        [ 1,  0,  1,  0],
        [-3,  0,  1,  1]],

       [[-3,  0,  0,  0],
        [-1,  0, -1,  0],
        [ 1,  0,  0,  0],
        [ 3,  0, -1,  0]],

       [[-1,  0,  0,  0],
        [ 1,  0,  0,  1],
        [-3,  1,  0,  0],
        [-5,  1,  0,  1]],

       [[-5,  0,  0,  0],
        [-3,  0,  0, -1],
        [-1,  0,  0, -1],
        [ 1,  0,  0,  0]],

       [[-1,  0,  0,  0],
        [-3,  0,  1,  0],
        [-5,  1,  0,  0],
        [-7,  1,  1,  0]]])